In [1]:
#@title Setup app
!pip install tensorflow
!pip install opencv-python
!pip install tensorflow_hub
!pip install matplotlib
!pip install numpy
!pip install Pillow



Click "Restart Runtime" after the cell above has been executed.

The output video files have a rather low resolution, due to how the actual neural style transfer is handled. You can upscale the video files either by simple upscaling or by utilizing the [Image Super Resolution](https://github.com/idealo/image-super-resolution) package. If you want to use the latter, please run the cell below and click on "Restart Runtime" once it's done.

In [1]:
#@title Optional: Install the Image Super Resolution package
!pip install ISR
!pip install 'h5py==2.10.0' --force-reinstall


     |████████████████████████████████| 86.3 MB 38 kB/s 
     |████████████████████████████████| 449 kB 41.8 MB/s 
     |████████████████████████████████| 50 kB 2.2 MB/s 
     |████████████████████████████████| 3.8 MB 35.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2a1b57458ddc072356c1c6da00e1a2a24de62d5cd8df25aba4a827b90b6f331e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Success

In [12]:
#@title Run the app
import os
import cv2
import logging

import functools


import tensorflow as tf
import tensorflow_hub as hub

import PIL
import os.path
from PIL import Image


input_file_path = "testvid.mp4" #@param {type:"string"}
filter_path = "frame1.png" #@param {type:"string"}
output_max_size = 256 #@param {type:"number"}
resize_factor = 4 #@param {type:"number"}
use_superres = True #@param {type:"boolean"}

logging.basicConfig(level=logging.DEBUG, datefmt='%Y-%m-%d:%H:%M:%S', format='%(asctime)s | %(levelname)s | %(name)s | LINE %(lineno)d: %(message)s')
log = logging.getLogger(__name__)
log.debug('Logging initialized')


class FrameExtractor:
    def __init__(self, input_video_path:str) -> None:

        self.input_video_path:str = input_video_path

        self.input_video_filename:str = ''

        self.extracted_frames_path:str = ''

        log.debug('FrameExtractor initialized')

    def create_subfolder(self):
        path_string_length:str = len(self.input_video_path)

        self.input_video_filename = self.input_video_path[:path_string_length - 4] # delete file extension from string

        parent_dir = 'output/extracted_frames/'

        self.extracted_frames_path = os.path.join(parent_dir, self.input_video_filename)

        if not os.path.exists(self.extracted_frames_path):
            try:
                os.makedirs(self.extracted_frames_path)
                log.debug(f'Output path for resized frames: {self.extracted_frames_path}')
            except Exception as e:
                log.error(f'Output path for resized frames FAILED: {e}')



    def extract_frames(self) -> str:
                
        cap = cv2.VideoCapture(self.input_video_path)

        ret, frame = cap.read()
        
        if not ret:
            log.error(f'File is not available at: {self.input_video_path}')

        self.create_subfolder()
        
        cap = cv2.VideoCapture(self.input_video_path)

        file_count:int = 0

        while True:
            ret, image = cap.read()

            if not ret:
                break

            cv2.imwrite(os.path.join(self.extracted_frames_path, f"frame{file_count}.png"), image)     
            log.debug(f"{file_count} frames extracted")
            file_count += 1

        log.info(f"{file_count} images are extracted in {self.extracted_frames_path}.")

        return self.extracted_frames_path, self.input_video_filename


# FrameExtractor('1.mp4').extract_frames()

class NeuralStyleTransferHandler:
    def __init__(self, extracted_frames_path:str, input_video_filename:str) -> None:
        os.environ['CUDA_VISIBLE_DEVICES']='-1'    # disable gpu

        # Load TF-Hub module

        # TODO: check if downloaded and set up on first startup, load from local afterwards
        hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
        #hub_handle = 'models/arbv1'
        self.hub_module = hub.load(hub_handle)

        self.extracted_frames_path = extracted_frames_path
        self.input_video_filename = input_video_filename

        parent_dir = 'output/stylized_frames/'

        self.stylized_frames_path = os.path.join(parent_dir, input_video_filename)

        if not os.path.exists(self.stylized_frames_path):
            try:
                os.makedirs(self.stylized_frames_path)
                log.debug(f'Output path for stylized frames: {self.stylized_frames_path}')
            except Exception as e:
                log.error(f'Output path for stylized frames FAILED: {e}')

        log.debug('NeuralStyleTransferHandler initialized')


    # TODO
    # def create_subfolder(self):


    def crop_center(self, image):
        '''Returns a cropped square image.'''
        shape = image.shape
        new_shape = min(shape[1], shape[2])
        offset_y = max(shape[1] - shape[2], 0) // 2
        offset_x = max(shape[2] - shape[1], 0) // 2
        image = tf.image.crop_to_bounding_box(
            image, offset_y, offset_x, new_shape, new_shape)
        return image


    def img_scaler(self, image, max_dim = output_max_size):

        # Casts a tensor to a new type.
        original_shape = tf.cast(tf.shape(image)[:-1], tf.float32)

        # Creates a scale constant for the image
        scale_ratio = max_dim / max(original_shape)

        # Casts a tensor to a new type.
        new_shape = tf.cast(original_shape * scale_ratio * 1, tf.int32)

        # Resizes the image based on the scaling constant generated above
        return tf.image.resize(image, new_shape)


    @functools.lru_cache(maxsize=None)
    def load_image(self, image_url, image_size=(256, 256), preserve_aspect_ratio=True):
        '''Loads and preprocesses images.'''
        # Cache image file locally.
        #image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
        image_path = image_url
        # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
        img = tf.io.decode_image(
            tf.io.read_file(image_path),
            channels=3, dtype=tf.float32)
        

        #img = crop_center(img)
        img = self.img_scaler(img)

        #img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
        return img[tf.newaxis, ...]





    def stylize_batch(self, mode:str,  filter_path: str):

        foldername = self.extracted_frames_path
        files_in_folder = len([file for file in os.listdir(f'{foldername}/')])

        if mode == "stylize_by_all_filters":
            stylized_folder = f'output/stylized_frames/{self.input_video_filename}_all_filter'
        else: 
            stylized_folder = f'output/stylized_frames/{self.input_video_filename}'

        if not os.path.exists(stylized_folder):
            os.mkdir(stylized_folder)

        for i in range(files_in_folder):

            if mode == 'stylize_by_all_filters':

                #content_image_url: str = f'{self.extracted_frames_path}/frame{place_in_folder}.png'
                style_image_url: str = f'filter/{i}.jpg'  
            
            elif mode=='stylize_by_filter':
                content_image_url: str = f'{self.extracted_frames_path}/frame{i}.png'
                style_image_url: str = filter_path
            
            output_image_size: int = 512


            # The content image size can be arbitrary.
            content_img_size = (output_image_size, output_image_size)
            # The style prediction model was trained with image size 256 and it's the 
            # recommended image size for the style image (though, other sizes work as 
            # well but will lead to different results).
            style_img_size = (256, 256)  # Recommended to keep it at 256.

            content_image = self.load_image(content_image_url, content_img_size)
            style_image = self.load_image(style_image_url, style_img_size)

            style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')


            outputs = self.hub_module(tf.constant(content_image), tf.constant(style_image))

            stylized_image = outputs[0]
            squeezed_image = tf.squeeze(stylized_image)
            
            tf.keras.preprocessing.image.save_img(f'{self.stylized_frames_path}/frame{i}.png', squeezed_image)
            log.debug(f"{i} frames stylized")

        return self.stylized_frames_path

class ImageResizer:
    def __init__(self, styled_frames_path:str, input_video_filename:str) -> None:

        self.styled_frames_path = styled_frames_path
        self.files_in_folder = len([file for file in os.listdir(f'{self.styled_frames_path}/')])

        self.input_video_filename = input_video_filename
        
        self.resized_frames_path = ''

        self.create_subfolder()


    def create_subfolder(self):

        # path_string_length:str = len(self.input_video_filename)

        # self.input_video_filename = self.input_video_filename[:path_string_length - 4] # delete file extension from string

        parent_dir = 'output/resized_frames/'

        self.resized_frames_path = os.path.join(parent_dir, self.input_video_filename)

        if not os.path.exists(self.resized_frames_path):
            try:
                os.makedirs(self.resized_frames_path)
                log.debug(f'Output path for resized frames: {self.resized_frames_path}')
            except Exception as e:
                log.error(f'Output path for resized frames FAILED: {e}')


    def resize(self, resize_factor=2):

        for file_count in range(self.files_in_folder):
            image_path = self.styled_frames_path + '/' + f'frame{file_count}.png'

            img = Image.open(image_path)
            img = img.resize((img.size[0] * resize_factor, img.size[1] * resize_factor)) # (width, height)

            img.save(self.resized_frames_path + '/' + f'frame{file_count}.png')
            
            log.debug(f"{file_count} frames resized")

        return self.resized_frames_path





    def resize_superres(self, resize_factor):

        import numpy as np
        # import sys
        # sys.path.append('..')
        from ISR.models import RDN, RRDN

        #model = RDN(weights='noise-cancel')
        #model = RRDN(weights='gans')
        model = RDN(weights='psnr-small')
        #model = RDN(weights='psnr-large')

        for file_count in range(self.files_in_folder):
            image_path = self.styled_frames_path + '/' + f'frame{file_count}.png'

            img = Image.open(image_path)

            img.resize(size=(img.size[0]*resize_factor, img.size[1]*resize_factor), resample=Image.BICUBIC)
            sr_img = model.predict(np.array(img))

            new_image = Image.fromarray(sr_img)
            new_image.save(f'{self.resized_frames_path}/frame{file_count}.png')
            
            log.debug(log.debug(f"{file_count} frames resized"))





class FrameCombiner:
    def __init__(self, styled_frames_path:str, input_video_filename:str) -> None:
        
        self.styled_frames_path:str = styled_frames_path
        self.input_video_filename:str = input_video_filename
        
        log.debug('FrameCombiner initialized')
        

    def combine_frames(self, filtername):

        img = cv2.imread(f'{self.styled_frames_path}/frame0.png', 0)

        # choose codec according to format needed
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        video = cv2.VideoWriter(f'{self.input_video_filename}_stylized_filter{filtername}.mp4', fourcc, 24, (img.shape[1], img.shape[0]))

        files_in_folder = len([file for file in os.listdir(f'{self.styled_frames_path}/')])

        for frame_count in range(0, files_in_folder):
            img = cv2.imread(f'{self.styled_frames_path}/frame{frame_count}.png')
            video.write(img)
            log.debug(f'{frame_count} frames combined')

        cv2.destroyAllWindows()
        video.release()


def main():

    #global filter_path, image_file_path

    # extract frames in folder
    extracted_frames_path, input_video_filename = FrameExtractor(input_file_path).extract_frames()

    # nst all frames in folder
    styled_frames_path = NeuralStyleTransferHandler(extracted_frames_path, input_video_filename).stylize_batch('stylize_by_filter', filter_path)

    # styled_frames_path = NeuralStyleTransferHandler(extracted_frames_path, input_video_filename).stylize_batch('stylize_by_filter', place_in_folder=filtername)
    # styled_frames_path = NeuralStyleTransferHandler(extracted_frames_path='output/extracted_frames/1', input_video_filename='1').stylize_batch('stylize_by_filter', place_in_folder=70)


    # optional: resize all frames in folder
    if resize_factor is not 1:
        if use_superres is True:
            resized_frames_path = ImageResizer(styled_frames_path, input_video_filename).resize_superres(resize_factor)
        else:
            resized_frames_path = ImageResizer(styled_frames_path, input_video_filename).resize(resize_factor)


    # combine all frames in video file
    FrameCombiner(styled_frames_path, input_video_filename).combine_frames(filter_path)
    # test = FrameCombiner(resized_frames_path, input_video_filename).combine_frames(filter_path)
    #test = FrameCombiner('output/resized_frames/2', '2').combine_frames(filter_path)


main()


2021-12-30:10:55:34 | DEBUG | __main__ | LINE 25: Logging initialized
2021-12-30:10:55:34 | DEBUG | __main__ | LINE 37: FrameExtractor initialized
2021-12-30:10:55:34 | DEBUG | __main__ | LINE 79: 0 frames extracted
2021-12-30:10:55:34 | DEBUG | __main__ | LINE 79: 1 frames extracted
2021-12-30:10:55:34 | DEBUG | __main__ | LINE 79: 2 frames extracted
2021-12-30:10:55:34 | DEBUG | __main__ | LINE 79: 3 frames extracted
2021-12-30:10:55:35 | DEBUG | __main__ | LINE 79: 4 frames extracted
2021-12-30:10:55:35 | DEBUG | __main__ | LINE 79: 5 frames extracted
2021-12-30:10:55:35 | INFO | __main__ | LINE 82: 6 images are extracted in output/extracted_frames/testvid.
2021-12-30:10:55:38 | DEBUG | __main__ | LINE 114: NeuralStyleTransferHandler initialized
2021-12-30:10:55:41 | DEBUG | __main__ | LINE 215: 0 frames stylized
2021-12-30:10:55:42 | DEBUG | __main__ | LINE 215: 1 frames stylized
2021-12-30:10:55:44 | DEBUG | __main__ | LINE 215: 2 frames stylized
2021-12-30:10:55:46 | DEBUG | __ma

True
superres


2021-12-30:10:55:50 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IHDR' 16 13
2021-12-30:10:55:50 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IDAT' 41 65536
2021-12-30:10:55:55 | DEBUG | __main__ | LINE 291: 0 frames resized
2021-12-30:10:55:55 | DEBUG | __main__ | LINE 291: None
2021-12-30:10:55:55 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IHDR' 16 13
2021-12-30:10:55:55 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IDAT' 41 65536
2021-12-30:10:56:01 | DEBUG | __main__ | LINE 291: 1 frames resized
2021-12-30:10:56:01 | DEBUG | __main__ | LINE 291: None
2021-12-30:10:56:01 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IHDR' 16 13
2021-12-30:10:56:01 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IDAT' 41 65536
2021-12-30:10:56:06 | DEBUG | __main__ | LINE 291: 2 frames resized
2021-12-30:10:56:06 | DEBUG | __main__ | LINE 291: None
2021-12-30:10:56:06 | DEBUG | PIL.PngImagePlugin | LINE 153: STREAM b'IHDR' 16 13
2021-12-30:10:56:06 | DEBUG | PIL.PngImagePlu

In [ ]:
#@title Download all output as a zip file
!zip -r ./output.zip ./output

from google.colab import files
files.download("output.zip")

In [7]:
#@title Remove output folder
%rm -r output